In [57]:
import os
import tensorflow as tf
import glob
import pathlib
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.layers import *
from keras.models import *

In [58]:
test=r"/kaggle/input/vegetable-image-dataset/Vegetable Images/test"
train=r"/kaggle/input/vegetable-image-dataset/Vegetable Images/train"
valid=r"/kaggle/input/vegetable-image-dataset/Vegetable Images/validation"

we use validation dataset and train our CNN model along with train dataset which will help us in hypertuning, hyperturning means finding the best values of batch_size learning rate etc.


In [59]:
def process(data):
    path=pathlib.Path(data)
    filepaths=list(path.glob(r"*/*.jpg"))
    labels=list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))
    df1=pd.Series(filepaths,name='filepaths').astype(str)
    df2=pd.Series(labels,name='labels')
    df=pd.concat([df1,df2],axis=1)
    return df

In [60]:
df_train=process(train)
df_test=process(test)
df_valid=process(valid)
        

Here we are using preprocess_input for data augmentation, we does Data augmentation in order to tackle over fitting and less accuracy issues

In [61]:
train_generator=ImageDataGenerator( preprocessing_function=preprocess_input)
test_generator=ImageDataGenerator( preprocessing_function=preprocess_input)
valid_generator=ImageDataGenerator( preprocessing_function=preprocess_input)

In [62]:
train_image=train_generator.flow_from_dataframe(dataframe=df_train,
                                                x_col='filepaths',
                                                y_col='labels',
                                                target_size=(224,224),
                                                batch_size=16
                                                )

test_image = test_generator.flow_from_dataframe(
    dataframe=df_test,
    x_col='filepaths',
    y_col='labels',
    target_size=(224,224),
    batch_size=16
)

valid_image = test_generator.flow_from_dataframe(
    dataframe=df_valid,
    x_col='filepaths',
    y_col='labels',
    target_size=(224,224),
    batch_size=16
)

Found 15000 validated image filenames belonging to 15 classes.
Found 3000 validated image filenames belonging to 15 classes.
Found 3000 validated image filenames belonging to 15 classes.


In [63]:
model_1 = keras.Sequential()

model_1.add(
    Conv2D(filters=20, kernel_size=(3, 3), activation="relu", input_shape=(224, 224, 3))
)
model_1.add(Conv2D(20, (3, 3), activation="relu"))
model_1.add(MaxPool2D((2, 2), padding="valid"))

model_1.add(Conv2D(15, (3, 3), activation="relu"))
model_1.add(Conv2D(15, (3, 3), activation="relu"))
model_1.add(MaxPool2D((2, 2)))

model_1.add(Flatten())
model_1.add(Dense(15, activation="softmax"))  # number of neurons have to match number of class

# Compile the model
model_1.compile(
    loss="categorical_crossentropy",
    optimizer='adam',
    metrics=["accuracy"],
)

# Fit the model
history_1 = model_1.fit(
        train_image,
        validation_data=valid_image,
        epochs=2,
)

Epoch 1/2
938/938 [==============================] - 1085s 1s/step - loss: 3.5260 - accuracy: 0.3309 - val_loss: 1.4365 - val_accuracy: 0.5423
Epoch 2/2
938/938 [==============================] - 1092s 1s/step - loss: 1.0971 - accuracy: 0.6565 - val_loss: 1.0105 - val_accuracy: 0.6773


In [64]:
model_1.evaluate(test_image)

188/188 [==============================] - 55s 292ms/step - loss: 1.0093 - accuracy: 0.6720


[1.0093165636062622, 0.671999990940094]